In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)


In [3]:
city_weather = pd.read_csv("../WeatherPy/w_output_data/city_weather.csv")
city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Busselton,-33.6500,115.3333,49.08,72,100,27.18,AU,1627597043
1,Geraldton,-28.7667,114.6000,64.13,88,40,16.11,AU,1627597043
2,Saint Paul Harbor,57.7900,-152.4072,56.71,67,90,6.91,US,1627597044
3,Yellowknife,62.4560,-114.3525,70.32,49,40,12.66,CA,1627596926
4,Cidreira,-30.1811,-50.2056,47.21,65,6,9.82,BR,1627597044
...,...,...,...,...,...,...,...,...,...
538,Menongue,-14.6585,17.6910,51.08,37,6,5.23,AO,1627597186
539,Qui Nhon,13.7667,109.2333,83.64,71,60,14.67,VN,1627597186
540,Siocon,7.7061,122.1353,79.14,90,100,4.61,PH,1627597187
541,Rāmhormoz,31.2800,49.6035,96.44,11,0,4.12,IR,1627597187


In [4]:
humidity = city_weather["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = city_weather[["Lat", "Lng"]]

In [6]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
criteria_city_df = city_weather.loc[(city_weather["Max Temp"] > 70) & (city_weather["Max Temp"] < 80) & (city_weather["Cloudiness"] == 0) & (city_weather["Wind Speed"] <= 10), :]
criteria_city_df = criteria_city_df.dropna(how='any')
criteria_city_df.reset_index(inplace=True)
del criteria_city_df['index']
criteria_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Changji,44.0167,87.3167,77.79,38,0,6.71,CN,1627597059
1,Dongsheng,39.8161,109.9776,71.42,38,0,5.17,CN,1627597120
2,Kalemie,-5.9475,29.1947,70.07,54,0,3.00,CD,1627597132
3,Bani Walid,31.7566,13.9942,76.71,55,0,6.62,LY,1627597133
4,Riachão do Jacuípe,-11.8069,-39.3856,75.63,59,0,7.58,BR,1627597134
5,Arroyo de la Encomienda,41.6096,-4.7969,74.59,56,0,9.22,ES,1627597135
6,Birjand,32.8663,59.2211,75.67,23,0,6.91,IR,1627597145
7,Shingū,33.7333,135.9833,78.91,70,0,3.36,JP,1627597153
8,Aripuanã,-9.1667,-60.6333,74.98,40,0,2.75,BR,1627597163
9,Sfântu Gheorghe,45.8667,25.7833,72.27,72,0,1.99,RO,1627597169


In [8]:
hotel_df = pd.DataFrame(criteria_city_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df

,City,Country,Lat,Lng
0,Changji,CN,44.0167,87.3167
1,Dongsheng,CN,39.8161,109.9776
2,Kalemie,CD,-5.9475,29.1947
3,Bani Walid,LY,31.7566,13.9942
4,Riachão do Jacuípe,BR,-11.8069,-39.3856
5,Arroyo de la Encomienda,ES,41.6096,-4.7969
6,Birjand,IR,32.8663,59.2211
7,Shingū,JP,33.7333,135.9833
8,Aripuanã,BR,-9.1667,-60.6333
9,Sfântu Gheorghe,RO,45.8667,25.7833


In [19]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "Hotel",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df



,City,Country,Lat,Lng,Hotel Name
0,Changji,CN,44.0167,87.3167,Ürümqi
1,Dongsheng,CN,39.8161,109.9776,Ordos City
2,Kalemie,CD,-5.9475,29.1947,Kalemie
3,Bani Walid,LY,31.7566,13.9942,Bani Waled
4,Riachão do Jacuípe,BR,-11.8069,-39.3856,Riachão do Jacuípe
5,Arroyo de la Encomienda,ES,41.6096,-4.7969,Valladolid
6,Birjand,IR,32.8663,59.2211,Birjand
7,Shingū,JP,33.7333,135.9833,Shingu
8,Aripuanã,BR,-9.1667,-60.6333,Aripuanã
9,Sfântu Gheorghe,RO,45.8667,25.7833,Sfântu Gheorghe


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))